Giải thuật Simulated Annealing

In [4]:
import math
import random

# Ma trận khoảng cách
graph = [
    [0, 10, 15, 20, 25],
    [10, 0, 35, 25, 30],
    [15, 35, 0, 30, 5],
    [20, 25, 30, 0, 15],
    [25, 30, 5, 15, 0]
]
n = len(graph)

def calculate_dist(route):
    d = 0
    for i in range(n):
        d += graph[route[i]][route[(i + 1) % n]]
    return d

def simulated_annealing():
    # 1. Khởi tạo lộ trình ngẫu nhiên
    current_route = list(range(n))
    random.shuffle(current_route)
    current_dist = calculate_dist(current_route)

    best_route = current_route[:]
    best_dist = current_dist

    # Các tham số nhiệt độ
    T = 1000.0          # Nhiệt độ ban đầu
    cooling_rate = 0.995 # Tốc độ hạ nhiệt
    T_min = 0.01        # Nhiệt độ dừng

    while T > T_min:
        # 2. Tạo một lộ trình láng giềng (hoán đổi 2 thành phố)
        new_route = current_route[:]
        i, j = random.sample(range(n), 2)
        new_route[i], new_route[j] = new_route[j], new_route[i]

        new_dist = calculate_dist(new_route)
        delta_e = new_dist - current_dist

        # 3. Quyết định có chấp nhận lộ trình mới không
        if delta_e < 0: # Tốt hơn -> Chấp nhận ngay
            current_route = new_route[:]
            current_dist = new_dist
            if current_dist < best_dist:
                best_dist = current_dist
                best_route = new_route[:]
        else:
            # Tệ hơn -> Chấp nhận dựa trên xác suất (Boltzmann distribution)
            if math.exp(-delta_e / T) > random.random():
                current_route = new_route[:]
                current_dist = new_dist

        # 4. Hạ nhiệt
        T *= cooling_rate

    return best_route, best_dist

# Chạy giải thuật
final_route, final_dist = simulated_annealing()
print(f"Lộ trình tìm được: {final_route}")
print(f"Tổng quãng đường: {final_dist}")

Lộ trình tìm được: [3, 4, 2, 0, 1]
Tổng quãng đường: 70


In [ ]:
Giải thuật di truyền GA

In [1]:
import numpy as np
import random

# 1. Khởi tạo ma trận khoảng cách (Ví dụ 5 thành phố)
distances = np.array([
    [0, 10, 15, 20, 25],
    [10, 0, 35, 25, 30],
    [15, 35, 0, 30, 5],
    [20, 25, 30, 0, 15],
    [25, 30, 5, 15, 0]
])

num_cities = len(distances)
pop_size = 20        # Quy mô quần thể
generations = 100    # Số thế hệ tiến hóa
mutation_rate = 0.1  # Tỉ lệ đột biến

# 2. Tính tổng quãng đường của một lộ trình
def calculate_fitness(route):
    distance = 0
    for i in range(num_cities):
        distance += distances[route[i]][route[(i + 1) % num_cities]]
    return 1 / distance  # Quãng đường càng ngắn, fitness càng cao

# 3. Tạo quần thể ban đầu ngẫu nhiên
def create_population():
    pop = []
    for _ in range(pop_size):
        route = list(range(num_cities))
        random.shuffle(route)
        pop.append(route)
    return pop

# 4. Lai ghép (Ordered Crossover - Đảm bảo con không bị lặp thành phố)
def crossover(parent1, parent2):
    start, end = sorted(random.sample(range(num_cities), 2))
    child = [None] * num_cities
    child[start:end] = parent1[start:end]

    pointer = 0
    for city in parent2:
        if city not in child:
            while child[pointer] is not None:
                pointer += 1
            child[pointer] = city
    return child

# 5. Đột biến (Swap Mutation - Đổi chỗ 2 thành phố ngẫu nhiên)
def mutate(route):
    if random.random() < mutation_rate:
        i, j = random.sample(range(num_cities), 2)
        route[i], route[j] = route[j], route[i]
    return route

# --- CHƯƠNG TRÌNH CHÍNH ---
population = create_population()

for gen in range(generations):
    # Sắp xếp quần thể dựa trên độ thích nghi
    population = sorted(population, key=lambda r: calculate_fitness(r), reverse=True)

    new_population = population[:2] # Giữ lại 2 cá thể tốt nhất (Elitism)

    while len(new_population) < pop_size:
        # Chọn lọc (Tournament Selection đơn giản)
        p1, p2 = random.sample(population[:10], 2)

        # Lai ghép
        child = crossover(p1, p2)

        # Đột biến
        child = mutate(child)

        new_population.append(child)

    population = new_population

# Kết quả cuối cùng
best_route = population[0]
best_distance = 1 / calculate_fitness(best_route)

print(f"Lộ trình tốt nhất tìm được: {best_route}")
print(f"Tổng quãng đường: {best_distance}")

Lộ trình tốt nhất tìm được: [1, 0, 2, 4, 3]
Tổng quãng đường: 70.0


Giải thuật quay lui (backtracking)

In [2]:
import sys

# Ma trận khoảng cách từ bài trước
graph = [
    [0, 10, 15, 20, 25],
    [10, 0, 35, 25, 30],
    [15, 35, 0, 30, 5],
    [20, 25, 30, 0, 15],
    [25, 30, 5, 15, 0]
]

n = len(graph)
visited = [False] * n
visited[0] = True # Xuất phát từ thành phố 0
min_distance = sys.maxsize
best_path = []

def backtracking_tsp(curr_city, count, current_dist, path):
    global min_distance, best_path

    # Bước dừng: Nếu đã đi qua tất cả n thành phố
    if count == n:
        # Cộng thêm quãng đường từ thành phố cuối cùng về lại thành phố 0
        total_dist = current_dist + graph[curr_city][0]
        if total_dist < min_distance:
            min_distance = total_dist
            best_path = path + [0]
        return

    # Duyệt qua các thành phố tiếp theo
    for next_city in range(n):
        if not visited[next_city]:
            # Đánh dấu đã thăm
            visited[next_city] = True

            # Đệ quy đi sâu vào nhánh này
            backtracking_tsp(
                next_city,
                count + 1,
                current_dist + graph[curr_city][next_city],
                path + [next_city]
            )

            # QUAY LUI: Bỏ đánh dấu để thử nhánh khác
            visited[next_city] = False

# Chạy giải thuật
backtracking_tsp(0, 1, 0, [0])

print(f"Lộ trình ngắn nhất: {' -> '.join(map(str, best_path))}")
print(f"Tổng quãng đường: {min_distance}")

Lộ trình ngắn nhất: 0 -> 1 -> 3 -> 4 -> 2 -> 0
Tổng quãng đường: 70


Giải thuật Hill Climbing

In [3]:
import random

# Ma trận khoảng cách
graph = [
    [0, 10, 15, 20, 25],
    [10, 0, 35, 25, 30],
    [15, 35, 0, 30, 5],
    [20, 25, 30, 0, 15],
    [25, 30, 5, 15, 0]
]
n = len(graph)

def calculate_total_distance(route):
    dist = 0
    for i in range(n):
        dist += graph[route[i]][route[(i + 1) % n]]
    return dist

def get_best_neighbor(current_route):
    best_dist = calculate_total_distance(current_route)
    best_neighbor = current_route[:]

    # Thử tất cả các cách hoán đổi 2 thành phố (tạo lân cận)
    for i in range(n):
        for j in range(i + 1, n):
            neighbor = current_route[:]
            neighbor[i], neighbor[j] = neighbor[j], neighbor[i] # Hoán đổi

            current_neighbor_dist = calculate_total_distance(neighbor)
            if current_neighbor_dist < best_dist:
                best_dist = current_neighbor_dist
                best_neighbor = neighbor[:]

    return best_neighbor, best_dist

def hill_climbing():
    # 1. Khởi tạo lộ trình ngẫu nhiên
    current_route = list(range(n))
    random.shuffle(current_route)
    current_dist = calculate_total_distance(current_route)

    print(f"Lộ trình ban đầu: {current_route} | Quãng đường: {current_dist}")

    while True:
        # 2. Tìm láng giềng tốt nhất
        neighbor, neighbor_dist = get_best_neighbor(current_route)

        # 3. Nếu láng giềng tốt hơn thì cập nhật, nếu không thì thoát (đã đạt đỉnh)
        if neighbor_dist < current_dist:
            current_dist = neighbor_dist
            current_route = neighbor[:]
            print(f"Cập nhật: {current_route} | Quãng đường: {current_dist}")
        else:
            break

    return current_route, current_dist

# Chạy giải thuật
final_route, final_dist = hill_climbing()
print("-" * 30)
print(f"Kết quả cuối cùng: {final_route}")
print(f"Tổng quãng đường ngắn nhất tìm được: {final_dist}")

Lộ trình ban đầu: [0, 4, 1, 3, 2] | Quãng đường: 125
Cập nhật: [4, 0, 1, 3, 2] | Quãng đường: 95
Cập nhật: [2, 0, 1, 3, 4] | Quãng đường: 70
------------------------------
Kết quả cuối cùng: [2, 0, 1, 3, 4]
Tổng quãng đường ngắn nhất tìm được: 70
